# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Feb 24th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

In [1]:
# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd

# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd
/data/pt_02586/sub_01/preprocessing


: 1

In [2]:
# We use from now on and forever the NEW bvec
# that were created by the EDDY command
# because they contain the small rotation
# that were introduced by motion correction
BVEC=$SUBJECTDIR'preprocessing/dwi_eddy.eddy_rotated_bvecs';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';

# We use from now on and forever the NEW bvec
# that were created by the EDDY command
# because they contain the small rotation
# that were introduced by motion correction
BVEC=$SUBJECTDIR'preprocessing/dwi_eddy.eddy_rotated_bvecs';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';


: 1

In [4]:
# In pratice, we have the MNI template
MNIFA='/afs/cbs.mpg.de/software/fsl/6.0.3/ubuntu-bionic-amd64/data/standard/FSL_HCP1065_FA_1mm.nii.gz'

# In pratice, we have the MNI template
ta/standard/FSL_HCP1065_FA_1mm.nii.gz'0.3/ubuntu-bionic-amd64/dat


: 1

### Diffusion Local Modeling

- The diffusion signal intensity is inversely proportional to the "amount of diffusion" in a given direction.  
- In our protocol, We have a lot of diffusion directions to "sample" this signal on the sphere.  

This signal on the sphere can be transformed into a **diffusion orientation distribution function (dODF)**, typically represented as a deformed sphere which represent the "amount of diffusion" at all orientations.  

- There is more diffusion along the orientation of WM bundles then across because it is easier (less obstacles) for water molecule to move in that directions.  
- Therefore, the maximas of the dODF should match bundle orienation.  
- However, there is still some diffusion in other orientations.  

This is why we want to create a **fiber orientation distribution function (fODF or FOD)** which is an object representing the "probabilities" or "proportion" of fiber at all orientation instead.  

This is very similar to the dODF but with sharper peak and it is therefore the prefered object for tractography.  

The idea is to estimate what the **dODF** looks like for voxel with only a single very well alligned bundle (no crossing, no fanning).  
We then use that special single bundle dODF (called response function) to **deconvolve** all the dODF in the brain and therefore compute de fODF. You can think of decovolution as a de-blurring on the sphere.  

We use "dwi2response" and "dwi2fod" from MRTRIX to estimate the response function and compute the fODF. 
This model is called constrained spherical deconvolution (CSD).  

And important detail of CSD is that it first fits a spherical harmonics (SH) to represent the signal, instead of working directly on the discrete bvec samples. We have to set a maximum order for the SH basis (-lmax) which will depend on the number of bvecs we have.  
The most common choice is lmax=8 (or sh order 8) but this requires 45+ bvecs to be well defined (i.e. the SH basis of order maximum 8 has 45 degree-of-freedom).  
Here we have 30 bvecs, we will therefore use lmax=6 (28 dof).
The generic formula for the number of dof of SH basis lmax is $$\frac{(l_\max+1)(l_\max+2)}{2}$$  

But in practice, you wouldnt want to use more than lmax=8 even if you have tons bvecs and you wouldnt be using CSD if you have less than 30 bvecs, so it's pretty much always order 8 or 6.  


In [ ]:
# command to estimate the single-fiber response function
# we use the fa method here which is simple and works well for in-vivo single bvalue
# This command will find the highest FA voxel inside a mask, align them and fit the SH.
# parameters:
# diffusion data
# file name to store the response function
# -fslgrad: bvecs and bvals
# -mask: (we dont want to include "artefact" voxels outside the brain as potential candidate)
# Here the mask is a tight brain mask with an FA threshold
# -voxels: file name to save a mask of which voxels where selected as single fiber population
# -lmax: SH basis order for the response function, this should always be 4
# -ntheads: number of cpu for faster computation
# -number: the number of single fiber voxel to use
dwi2response fa \
    dwi_eddy.nii.gz \
    response_fa.txt \
    -fslgrad $BVEC $BVAL \
    -mask fa_eddy_th0p3.nii.gz \
    -voxels response_voxel_fa.nii.gz \
    -lmax 4 \
    -nthreads 20 \
    -number 300;


In [ ]:
# You can visually inspect the response function with the following command:
shview response_fa.txt
# It should look like a puffy disk

In [ ]:
# command to compute csd on the brain
# we specify csd as the model since we are using single bvalue data
# parameters: 
# diffusion data
# the response function
# file name to save the FOD (it is saved as SH coefficients)
# -fslgrad: bvecs and bvals
# -mask: Here I am using a larger mask than for the response function (a BET mask)
# This is because I want to make sure I have an FOD everywhere, including low FA voxels
# (such has crossing or near cortex sometimes)
# -ntheads: number of cpu for faster computation
# -lmax: SH basis discused before, this depends on the number of bvec
dwi2fod csd \
    dwi_eddy.nii.gz \
    response_fa.txt \
    fod_lmax6.nii.gz \
    -fslgrad $BVEC $BVAL \
    -mask b0_eddy_avg_N4x3_bet_0p4_mask.nii.gz \
    -nthreads 20 \
    -lmax 6;
